# Зміст
- [Про Guidance](../../../../code/01.Introduce)
- [Налаштування](../../../../code/01.Introduce)
- [Незалежне Генерування](../../../../code/01.Introduce)
- [Говорячи за Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Вибір](../../../../code/01.Introduce)
- [Ланцюжок Міркувань](../../../../code/01.Introduce)
- [Генерування JSON](../../../../code/01.Introduce)
- [Генерування HTML](../../../../code/01.Introduce)


# Про Guidance
Guidance — це перевірена бібліотека з відкритим кодом для Python, яка дозволяє контролювати результати будь-якої мовної моделі (LM). За допомогою одного виклику API ви можете висловити (у Python) точні програмні обмеження, яких модель повинна дотримуватися, і створити структурований результат у форматі JSON, Python, HTML, SQL або будь-якій іншій структурі, необхідній для вашого випадку використання.

Guidance відрізняється від традиційних технік підказок. Він забезпечує дотримання обмежень, спрямовуючи модель токен за токеном на рівні інференції, що дозволяє отримувати результати вищої якості та знижувати витрати й затримки на 30–50% у випадках використання з високою структурованістю.

Щоб дізнатися більше про Guidance, відвідайте [публічний репозиторій на GitHub](https://github.com/guidance-ai/guidance) або перегляньте [сесію Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) на Microsoft Build.


# Налаштування
1. Встановіть Guidance за допомогою `pip install guidance --pre`
2. Розгорніть кінцеву точку Phi 3.5 mini в Azure, перейшовши за посиланням https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml і натиснувши кнопку "Deploy"
3. Збережіть API-ключ вашої кінцевої точки в змінній середовища під назвою `AZURE_PHI3_KEY`, а URL-адресу в змінній середовища під назвою `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Неконтрольована генерація
Текст можна генерувати без жодних обмежень за допомогою функції `gen()`. Це те саме, що використовувати модель без Guidance.

## Форматування чату
Як і багато інших моделей чату, Phi-3 очікує повідомлення між користувачем і асистентом у певному форматі. Guidance підтримує шаблон чату Phi-3 і автоматично керує форматуванням чату для вас. Щоб створити черги чату, розміщуйте кожну частину розмови в блоці `with user()` або `with assistant()`. Блок `with system()` можна використовувати для встановлення системного повідомлення.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Економія токенів
У чітко структурованих сценаріях Guidance може пропускати токени та генерувати лише необхідні, що покращує продуктивність, підвищує ефективність і знижує витрати на API. Згенеровані токени в цьому зошиті відображаються з виділеним фоном. Примусові токени показані без виділення і коштують стільки ж, скільки й вхідні токени, які оцінюються як третина вартості вихідних токенів.

*Примітка:* У першому прикладі з неконтрольованою генерацією не вдалося примусово застосувати жоден токен, оскільки ми не надали жодних обмежень.


# Говоримо про Phi 3  
Завдяки Guidance, ви можете легко вставляти текст у відповіді моделі. Це може бути корисним, якщо ви хочете спрямувати вихід моделі в конкретному напрямку.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Обмеження за допомогою регулярних виразів
У попередньому прикладі Phi 3 відповів із додатковими поясненнями після того, як дав відповідь `Canberra`. Щоб обмежити вихід моделі до одного слова, можна використати регулярний вираз.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# Вибір з кількох варіантів
Коли відомі деякі можливі варіанти, ви можете використовувати функцію `select()`, щоб модель обрала з наданого списку опцій.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

З `select()`, був згенерований лише токен `Can`. Оскільки `Canberra` є єдиним варіантом, який може завершити відповідь, решта токенів були примусово додані.


# Ланцюжок міркувань
Ланцюжок міркувань — це техніка, яка може допомогти покращити якість результатів моделі, заохочуючи її розв’язувати проблему крок за кроком. Зазвичай, щоб дійти до остаточної відповіді, необхідно кілька етапів підказок. Спочатку потрібно дати інструкцію моделі мислити поетапно. Потім слід знову звернутися до моделі, щоб отримати остаточну відповідь. У стандартних API для чат-інференції це потребує 2 викликів API, і згенерований моделлю «ланцюжок міркувань» оплачується двічі – один раз як вихідні токени, коли модель його створила, і ще раз як вхідні токени для другого виклику. Завдяки Guidance весь багатокроковий процес обробляється та оплачується в межах одного виклику API, що знижує витрати та затримки.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Генерація JSON
Guidance можна використовувати для забезпечення створення JSON, який відповідає JSON-схемі або моделі pydantic, наприклад, схемі профілю користувача, показаній тут.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Генерація HTML

Guidance також можна використовувати для створення коду та дотримання синтаксичних вимог у мовах програмування. У цьому розділі ми створимо невелику програму Guidance для написання дуже простих HTML-сторінок.

Ми розділимо вебсторінку на менші секції, кожна з яких матиме свою функцію Guidance. Потім ці секції будуть об'єднані у нашій фінальній функції для створення HTML-сторінки. 
Після цього ми запустимо цю функцію на моделі з підтримкою Guidance в Azure AI.

*Примітка:* Це не буде повноцінний генератор HTML; мета полягає в тому, щоб показати, як можна створювати структурований вихідний матеріал для ваших індивідуальних потреб.

Ми починаємо з імпорту необхідного з Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML-сторінки мають чітку структуру, і ми будемо «примушувати» ці частини сторінки за допомогою Guidance.  
Коли ми явно вимагаємо текст від моделі, необхідно переконатися, що він не містить нічого, що могло б бути тегом, тобто ми повинні виключити символи '<' і '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Ми можемо використовувати цю функцію для створення тексту в межах довільного HTML-тега:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Тепер створимо заголовок сторінки. У рамках цього нам потрібно згенерувати заголовок сторінки:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Тіло HTML-сторінки буде заповнене заголовками та абзацами.  
Ми можемо визначити функцію для кожного:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Тепер функція для визначення основної частини самого HTML.  
Вона використовує `select()` з параметром `recurse=True` для створення кількох заголовків і абзаців:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Далі ми переходимо до функції, яка генерує повну HTML-сторінку.  
Ми додаємо початковий тег HTML, потім створюємо заголовок, потім тіло, і на завершення додаємо кінцевий тег HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Ми надаємо зручний обгортковий інтерфейс, який дозволить нам:
- Встановлювати температуру генерації
- Захоплювати згенеровану сторінку з об'єкта Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Ми можемо записати результат у файл:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

І [перегляньте результат](../../../../code/01.Introduce/sample_page.html).



---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, зверніть увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають у результаті використання цього перекладу.
